In [1]:
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
from pyspark.sql.types import *
from pyspark.sql.functions import lit



db_path = 'd:/dw/data-warehouse/stocklab_db'
appName = "test_spark"
master = "local[4]"


#conf = SparkConf().setAll([("spark.driver.maxResultSize", '6g'), ('spark.executor.cores', '1'), ('spark.submit.deployMode', 'cluster')])

spark = SparkSession.builder.appName(appName) \
  .config("spark.sql.warehouse.dir", db_path) \
  .config("spark.jars.packages", "io.delta:delta-core_2.12:0.8.0") \
  .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
  .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
  .enableHiveSupport() \
  .getOrCreate()

from delta.tables import *
from pyspark.sql.functions import *

In [2]:
#spark.sql("drop database if exists stocklab CASCADE")
#AnalysisException: org.apache.hadoop.hive.ql.metadata.HiveException: InvalidOperationException(message:Database stocklab is not empty. One or more tables exist.)
#force drop cascade
spark.sql("create database IF NOT EXISTS stocklab")
spark.sql("show databases ").show()
spark.sql("describe database stocklab").show(100,False)

+---------+
|namespace|
+---------+
|  default|
| stocklab|
+---------+

+-------------------------+--------------------------------------------------+
|database_description_item|database_description_value                        |
+-------------------------+--------------------------------------------------+
|Database Name            |stocklab                                          |
|Comment                  |                                                  |
|Location                 |file:/d:/dw/data-warehouse/stocklab_db/stocklab.db|
|Owner                    |Administrator                                     |
+-------------------------+--------------------------------------------------+



# stock_min data load

- partition select 

In [8]:
# partition 정보를 통해 pushdown 할 수 있는 방법이 있는가? 실제 
df_min = spark.read.format("parquet").load("d:/dw/stock_min")

In [6]:
df_min.describe()

DataFrame[summary: string, close: string, date: string, high: string, jdiff_vol: string, low: string, open: string, time: string, value: string, shcode: string]

In [11]:
df_min.filter(df_min.date > '20210501').show()

+-----+----+---------+----+----+------+-----+--------+------+
|close|high|jdiff_vol| low|open|  time|value|    date|shcode|
+-----+----+---------+----+----+------+-----+--------+------+
| 4340|4430|   256689|4300|4430|143000| 1122|20210514| 61250|
| 4320|4395|   143971|4300|4345|143100|  626|20210514| 61250|
| 4290|4330|    81408|4250|4325|143200|  349|20210514| 61250|
| 4245|4290|    64016|4240|4290|143300|  273|20210514| 61250|
| 4250|4265|    38360|4225|4245|143400|  163|20210514| 61250|
| 4280|4285|    30856|4255|4255|143500|  132|20210514| 61250|
| 4365|4375|    94219|4280|4285|143600|  408|20210514| 61250|
| 4385|4405|   159427|4350|4370|143700|  698|20210514| 61250|
| 4400|4450|   242606|4380|4385|143800| 1071|20210514| 61250|
| 4520|4520|   374618|4400|4400|143900| 1673|20210514| 61250|
| 4520|4520|        0|4520|4520|144000|    0|20210514| 61250|
| 4520|4520|        0|4520|4520|144100|    0|20210514| 61250|
| 4575|4580|   769771|4475|4485|144200| 3474|20210514| 61250|
| 4610|4

In [12]:
df_min.explain()

== Physical Plan ==
*(1) ColumnarToRow
+- FileScan parquet [close#618,high#619,jdiff_vol#620,low#621,open#622,time#623,value#624,date#625,shcode#626] Batched: true, DataFilters: [], Format: Parquet, Location: InMemoryFileIndex[file:/d:/dw/stock_min], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<close:string,high:string,jdiff_vol:string,low:string,open:string,time:string,value:string>




In [14]:
df_pre = df_min.filter(df_min.date > '20210501')
df_pre.explain()

== Physical Plan ==
*(1) ColumnarToRow
+- FileScan parquet [close#618,high#619,jdiff_vol#620,low#621,open#622,time#623,value#624,date#625,shcode#626] Batched: true, DataFilters: [], Format: Parquet, Location: InMemoryFileIndex[file:/d:/dw/stock_min], PartitionFilters: [isnotnull(date#625), (date#625 > 20210501)], PushedFilters: [], ReadSchema: struct<close:string,high:string,jdiff_vol:string,low:string,open:string,time:string,value:string>




In [16]:
df_pre.explain(extended=True)

== Parsed Logical Plan ==
'Filter (date#625 > 20210501)
+- Relation[close#618,high#619,jdiff_vol#620,low#621,open#622,time#623,value#624,date#625,shcode#626] parquet

== Analyzed Logical Plan ==
close: string, high: string, jdiff_vol: string, low: string, open: string, time: string, value: string, date: int, shcode: string
Filter (date#625 > cast(20210501 as int))
+- Relation[close#618,high#619,jdiff_vol#620,low#621,open#622,time#623,value#624,date#625,shcode#626] parquet

== Optimized Logical Plan ==
Filter (isnotnull(date#625) AND (date#625 > 20210501))
+- Relation[close#618,high#619,jdiff_vol#620,low#621,open#622,time#623,value#624,date#625,shcode#626] parquet

== Physical Plan ==
*(1) ColumnarToRow
+- FileScan parquet [close#618,high#619,jdiff_vol#620,low#621,open#622,time#623,value#624,date#625,shcode#626] Batched: true, DataFilters: [], Format: Parquet, Location: InMemoryFileIndex[file:/d:/dw/stock_min], PartitionFilters: [isnotnull(date#625), (date#625 > 20210501)], PushedFilter

In [29]:
df_pre.rdd.getNumPartitions()

1937

In [30]:
df_pre.rdd.partitions.size()

AttributeError: 'RDD' object has no attribute 'partitions'

In [24]:
df_repart = df_pre.repartition(5)

In [25]:
df_repart.rdd.getNumPartitions()

5

# repartition

# collease


In [32]:
df_repart.explain(extended=True)

== Parsed Logical Plan ==
Repartition 5, true
+- Filter (date#625 > cast(20210501 as int))
   +- Relation[close#618,high#619,jdiff_vol#620,low#621,open#622,time#623,value#624,date#625,shcode#626] parquet

== Analyzed Logical Plan ==
close: string, high: string, jdiff_vol: string, low: string, open: string, time: string, value: string, date: int, shcode: string
Repartition 5, true
+- Filter (date#625 > cast(20210501 as int))
   +- Relation[close#618,high#619,jdiff_vol#620,low#621,open#622,time#623,value#624,date#625,shcode#626] parquet

== Optimized Logical Plan ==
Repartition 5, true
+- Filter (isnotnull(date#625) AND (date#625 > 20210501))
   +- Relation[close#618,high#619,jdiff_vol#620,low#621,open#622,time#623,value#624,date#625,shcode#626] parquet

== Physical Plan ==
Exchange RoundRobinPartitioning(5), REPARTITION_WITH_NUM, [id=#91]
+- *(1) ColumnarToRow
   +- FileScan parquet [close#618,high#619,jdiff_vol#620,low#621,open#622,time#623,value#624,date#625,shcode#626] Batched: true,